# GRB Classification with Gaussian Mixture Models

This notebook demonstrates how to use `grb_pipeline` to classify GRBs as short or long
using a Gaussian Mixture Model (GMM) in the T90-Hardness Ratio plane.

The data are taken from Goldstein et al. 2017 (ApJL, 848, L14).

Authors: Vikas Chand

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii
from grb_pipeline.analysis.classification import GRBClassifier, GRBClass, GMMResult

## 1. Load Fermi GBM Catalog Data

In [ ]:
# Read the Goldstein et al. 2017 catalog data
Gold_data = ascii.read('data_LogT90_LogH32.txt', header_start=None, data_start=1, comment='#')

logT90 = np.array(Gold_data['col1'])
logHR = np.array(Gold_data['col2'])

print(f"Loaded {len(logT90)} GRBs from Fermi GBM catalog")
print(f"log(T90) range: [{logT90.min():.2f}, {logT90.max():.2f}]")
print(f"log(HR) range: [{logHR.min():.2f}, {logHR.max():.2f}]")

In [ ]:
# Quick look at the raw data
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(logT90, logHR, s=3, alpha=0.5, color='gray')
ax.set_xlabel(r'$\log_{10}(T_{90}\ /\ \mathrm{s})$', fontsize=14)
ax.set_ylabel(r'$\log_{10}(\mathrm{Hardness\ Ratio})$', fontsize=14)
ax.set_title('Fermi GBM Catalog: T90 vs Hardness Ratio')
ax.set_xlim(-2, 4)
ax.set_ylim(-1.5, 1.5)
plt.tight_layout()
plt.show()

## 2. Fit a 2-Component Gaussian Mixture Model

We fit a 2-component GMM with full covariance to the (log T90, log HR) plane.
This separates the population into short-hard and long-soft clusters.

In [ ]:
classifier = GRBClassifier()

# Fit the GMM
result = classifier.fit_gmm(logT90, logHR, n_components=2, covariance_type='full')

print("GMM Fit Results:")
print(f"  Weights: {result.weights}")
print(f"  Means:\n{result.means}")
print(f"  Covariances:\n{result.covariances}")

## 3. Classification Plot with Probability Contours

The plot shows:
- Catalog GRBs coloured by their GMM cluster assignment
- Dashed contour lines at P(Short) = 0.25, 0.50, 0.75
- Background colour mesh showing the probability surface

In [ ]:
fig = classifier.plot_gmm_classification(
    logT90, logHR,
    grid_resolution=200,
    contour_levels=[0.25, 0.50, 0.75],
)
plt.show()

## 4. Classify a Specific GRB

Given a GRB's T90 and hardness ratio, we can determine the probability
of it being a short GRB.

In [ ]:
# Example: classify a GRB with T90=0.5s and HR=2.0
t90_example = 0.5
hr_example = 2.0

classification = classifier.classify_gmm(t90_example, hr_example)
prob_short = classifier.short_grb_probability(
    np.array([np.log10(t90_example)]),
    np.array([np.log10(hr_example)])
)[0]

print(f"GRB with T90={t90_example}s, HR={hr_example}")
print(f"  Classification: {classification.value}")
print(f"  P(Short GRB) = {prob_short:.3f}")

In [ ]:
# Plot with the target GRB highlighted
fig = classifier.plot_gmm_classification(
    logT90, logHR,
    target_grb=(np.log10(t90_example), np.log10(hr_example)),
    contour_levels=[0.25, 0.50, 0.75],
)
plt.show()